In [8]:
import numpy as np
import os
from os import listdir, makedirs
from os.path import isfile, join, exists
import matplotlib.pyplot as plt
import random
from skimage.util import random_noise
from skimage import img_as_ubyte
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
import cv2
import math
import sys


from clbp import getmapping, clbp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Copy Data from Google Drive
!cp -a "/content/drive/MyDrive/CURET.zip" "/content/CURET.zip"
!unzip "/content/CURET.zip"

!cp -a "/content/drive/MyDrive/UIUC.zip" "/content/UIUC.zip"
!unzip "/content/UIUC.zip"

Streaming output truncated to the last 5000 lines.
  inflating: CURET/sample19/19-002.png  
  inflating: CURET/sample19/19-011.png  
  inflating: CURET/sample19/19-014.png  
  inflating: CURET/sample19/19-016.png  
  inflating: CURET/sample19/19-017.png  
  inflating: CURET/sample19/19-019.png  
  inflating: CURET/sample19/19-022.png  
  inflating: CURET/sample19/19-025.png  
  inflating: CURET/sample19/19-026.png  
  inflating: CURET/sample19/19-028.png  
  inflating: CURET/sample19/19-030.png  
  inflating: CURET/sample19/19-032.png  
  inflating: CURET/sample19/19-033.png  
  inflating: CURET/sample19/19-035.png  
  inflating: CURET/sample19/19-036.png  
  inflating: CURET/sample19/19-037.png  
  inflating: CURET/sample19/19-039.png  
  inflating: CURET/sample19/19-041.png  
  inflating: CURET/sample19/19-042.png  
  inflating: CURET/sample19/19-043.png  
  inflating: CURET/sample19/19-044.png  
  inflating: CURET/sample19/19-045.png  
  inflating: CURET/sample19/19-046.png  
  infl

In [1]:
DataSetName = "CURET"#CURET/UIUC/Outex23

In [9]:
def loadCURET():
    FilePath = r"/content/CURET"
    dires = listdir(FilePath)
    label = 0
    trainFiles = [""]
    testFiles = [""]
    trainImages = np.empty([1,200,200])
    trainImagesLBPriu2 = np.zeros((1,648),dtype = float)
    trainLabels = np.array([])
    testImages = np.empty([1,200,200])
    testImagesLBPriu2 = np.zeros((1,648),dtype = float)
    testLabels = np.array([])
    random.seed(250)
    mapping16riu2 = getmapping(16,'riu2')
    for dir in dires:
        print(dir)
        trainCount = 0
        testCount = 0
        tempPath = join(FilePath,dir)
        images = [f for f in listdir(tempPath) if isfile(join(tempPath, f))]
        for image in images:
            if(image[-3:] != "png"):
                continue
            imageContent = cv2.cvtColor(cv2.imread(join(tempPath, image)),cv2.COLOR_BGR2GRAY)
            if imageContent is None:
                print("Error reading image: " + join(tempPath, image))
            clbp16riu2 = clbp(imageContent,R=2,P=16,patternMapping=mapping16riu2).reshape(1,648)
            imageContent = imageContent.reshape(1,200,200)
            randNumber = random.random()
            if trainCount == 46:
              randNumber = 0.6
            elif testCount == 46:
              randNumber = 0.4
            if randNumber <= 0.5 :
                trainImages = np.append(trainImages, imageContent,axis = 0)
                trainImagesLBPriu2 = np.append(trainImagesLBPriu2,clbp16riu2,axis = 0)
                trainLabels = np.append(trainLabels, label)
                trainCount += 1
                trainFiles.append(join(tempPath, image))
            else:
                testImages = np.append(testImages, imageContent,axis = 0)
                testImagesLBPriu2 = np.append(testImagesLBPriu2,clbp16riu2,axis = 0)
                testLabels = np.append(testLabels, label)
                testCount += 1
                testFiles.append(join(tempPath, image))
        label+=1
    trainImages = trainImages[1:]
    testImages = testImages[1:]
    trainImagesLBPriu2 = trainImagesLBPriu2[1:]
    testImagesLBPriu2 = testImagesLBPriu2[1:]
    trainImages = trainImages.reshape(trainImages.shape[0],trainImages.shape[1],trainImages.shape[2],1)
    testImages = testImages.reshape(testImages.shape[0],testImages.shape[1],testImages.shape[2],1)
    trainImages = np.concatenate((trainImages,trainImages,trainImages),axis=3)
    testImages = np.concatenate((testImages,testImages,testImages),axis=3)
    trainFiles = trainFiles[1:]
    testFiles = testFiles[1:]
    return (trainImages, trainImagesLBPriu2, trainLabels,trainFiles),(testImages, testImagesLBPriu2,testLabels,testFiles)

def loadUIUC():
    FilePath = r"/content/UIUCtex"
    dires = listdir(FilePath)
    label = 0
    trainImages = np.empty([1,200,267])
    trainImagesLBPriu2 = np.zeros((1,648),dtype = float)
    trainLabels = np.array([])
    testImages = np.empty([1,200,267])
    testImagesLBPriu2 = np.zeros((1,648),dtype = float)
    testLabels = np.array([])
    random.seed(250)
    mapping16riu2 = getmapping(16,'riu2')
    for dir in dires:
        print(dir)
        trainCount = 0
        testCount = 0
        tempPath = join(FilePath,dir)
        images = [f for f in listdir(tempPath) if isfile(join(tempPath, f))]
        for image in images:
            if(image[-3:] != "jpg"):
                continue
            imageContent = cv2.resize(cv2.cvtColor(cv2.imread(join(tempPath, image)),cv2.COLOR_BGR2GRAY),(267,200))
            if image is None:
                print("Error reading image: " + join(tempPath, image))
            clbp16riu2 = clbp(imageContent,R=2,P=16,patternMapping=mapping16riu2).reshape(1,648)#648

            imageContent = imageContent.reshape(1,200,267)
            randNumber = random.random()
            if trainCount == 20:
                randNumber = 0.6
            elif testCount == 20:
                randNumber = 0.4
            if randNumber <= 0.5 :
                trainImages = np.append(trainImages, imageContent,axis = 0)
                trainImagesLBPriu2 = np.append(trainImagesLBPriu2, clbp16riu2,axis = 0)
                trainLabels = np.append(trainLabels, label)
                trainCount += 1
            else:
                testImages = np.append(testImages, imageContent,axis = 0)
                testImagesLBPriu2 = np.append(testImagesLBPriu2, clbp16riu2,axis = 0)
                testLabels = np.append(testLabels, label)
                testCount += 1
        label+=1
    trainImages = trainImages[1:]
    testImages = testImages[1:]
    trainImagesLBPriu2 = trainImagesLBPriu2[1:]
    testImagesLBPriu2 = testImagesLBPriu2[1:]
    trainImages = trainImages.reshape(trainImages.shape[0],trainImages.shape[1],trainImages.shape[2],1)
    testImages = testImages.reshape(testImages.shape[0],testImages.shape[1],testImages.shape[2],1)
    trainImages = np.concatenate((trainImages,trainImages,trainImages),axis=3)
    testImages = np.concatenate((testImages,testImages,testImages),axis=3)

    return (trainImages, trainImagesLBPriu2, trainLabels),(testImages, testImagesLBPriu2,testLabels)

def loadOutex23():
    tests=[5,15,30,40,50]
    OutexTC00023basePath = r"/content/Outex-TC-00023/"
    trainFilePath = join(OutexTC00023basePath, "000/train.txt")
    testFilePath = join(OutexTC00023basePath, "000/train.txt")
    train_images_path = join(OutexTC00023basePath, "images")
    test_images_path = "/content/Outex-TC-00023/sp"
    mapping16riu2 = getmapping(16,'riu2')
    trainFile = open(trainFilePath, "r")
    trainFileContent = trainFile.readlines()
    trainImageCount = int(trainFileContent[0])
    trainImagesLBP = np.zeros((trainImageCount,648),dtype = float)
    trainImages = np.zeros((trainImageCount,128,128,1),dtype = float)
    trainLabels = np.zeros(trainImageCount,dtype = np.uint8)
    for i in range(1,trainImageCount+1):
        index = i-1
        currentFileName, currentClassLabel = trainFileContent[i].split(" ")
        imageContent = cv2.imread(join(train_images_path, currentFileName),cv2.IMREAD_GRAYSCALE)

        if image is None:
            print("Error reading image: " + join(train_images_path, currentFileName))

        clbp16riu2 = clbp(imageContent,R=2,P=16,patternMapping=mapping16riu2)#648
        trainImagesLBP[index] = clbp16riu2
        trainImages[index] = imageContent.reshape((128,128,1))
        trainLabels[index] = int(currentClassLabel)
    testFile = open(testFilePath, "r")
    testFileContent = testFile.readlines()
    testImageCount = int(testFileContent[0])
    testImagesLBP = np.zeros((5,testImageCount,648),dtype = np.float)
    testImages = np.zeros((5,testImageCount,128,128,1),dtype = np.float)
    testLabels = np.zeros((5,testImageCount),dtype = np.uint8)
    for testi in range(0,len(tests)):
      cur_test_images_path = test_images_path+'/'+str(tests[testi])
      for i in range(1,testImageCount+1):
          index = i-1
          print("test",index)
          currentFileName, currentClassLabel = testFileContent[i].split(" ")
          imageContent = cv2.imread(join(cur_test_images_path, currentFileName),cv2.IMREAD_GRAYSCALE)

          if image is None:
              print("Error reading image: " + join(cur_test_images_path, currentFileName))

          clbp16riu2 = clbp(imageContent,R=2,P=16,patternMapping=mapping16riu2)
          testImagesLBP[testi,index] = clbp16riu2

          testImages[testi,index] = imageContent.reshape((128,128,1))
          testLabels[testi,index] = int(currentClassLabel)

    trainImages = np.concatenate((trainImages,trainImages,trainImages),axis=3)
    testImages = np.concatenate((testImages,testImages,testImages),axis=4)

    return (trainImages, trainImagesLBP,trainLabels),(testImages, testImagesLBP, testLabels)



In [ ]:
if DataSetName == "CURET":
   (trainImages, trainImagesLBP, trainLabels,trainFiles),(testImages, testImagesLBP,testLabels,testFiles) = loadCURET()
   labelCount = 61
   inputShape = (200,200,3)
elif DataSetName == "UIUC":
   (trainImages, trainImagesLBP, trainLabels,trainFiles),(testImages, testImagesLBP,testLabels,testFiles) = loadUIUC()
   labelCount = 25
   inputShape = (200,267,3)
else:
   (trainImages, trainImagesLBP, trainLabels,trainFiles),(testImages, testImagesLBP,testLabels,testFiles) = loadOutex23()
   labelCount = 68
   inputShape = (128,128,3)

trainLBP162D = trainImagesLBP.reshape(trainImagesLBP.shape[0],18,36,1)
testLBP162D  = testImagesLBP.reshape(testImagesLBP.shape[0],18,36,1)
trainLBP162D = trainLBP162D/np.max(trainLBP162D)
testLBP162D = testLBP162D/np.max(testLBP162D)

trainLabels = tensorflow.keras.utils.to_categorical(trainLabels, labelCount)
testLabels = tensorflow.keras.utils.to_categorical(testLabels, labelCount)

T15_brick2
T04_wood1
T14_brick1
T02_bark2
T13_wall
T11_floor2
T07_water
T08_granite


In [ ]:
LbpInput2 = tensorflow.keras.Input(shape=(18,36,1),name="LbpInput2")
Lbpmodel2 = Conv2D(32,(5,5), activation='relu')(LbpInput2)
Lbpmodel2 = Conv2D(64,(3,3), activation='relu')(Lbpmodel2)
Lbpmodel2 = GlobalAveragePooling2D()(Lbpmodel2)


conv_Input = tensorflow.keras.Input(shape=inputShape,name="conv")
from tensorflow.keras.applications import DenseNet121
conv_model = DenseNet121(include_top=False,weights="imagenet",input_shape=inputShape,pooling='avg')
conv_model = conv_model(conv_Input)

output = layers.Concatenate()([conv_model,Lbpmodel2])

Lbpmodel = Dense(1000, activation='relu')(output)
Lbpmodel = Dropout(0.1)(Lbpmodel)
Lbpmodel = Dense(labelCount, activation='softmax')(Lbpmodel)
Lbpfinalmodel = Model(inputs=[conv_Input,LbpInput2], outputs=Lbpmodel)
Lbpfinalmodel.summary()

initial_learning_rate = 0.05
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=500,
    decay_rate=0.96,
    staircase=True)
Lbpfinalmodel.compile(optimizer=tensorflow.keras.optimizers.SGD(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=["accuracy"])


In [ ]:
result = Lbpfinalmodel.fit({"conv":trainImages,"LbpInput2":trainLBP162D}, trainLabels,batch_size=16, epochs=30,verbose=1,validation_data=({"conv":testImages,"LbpInput2":testLBP162D},testLabels))